In [1]:
import requests
import json
import re
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [50]:
'''
Some of the product_names in the dataframe were random (e.g. Okay?) and led to nowhere. 
So all the rows corresponding to such titles were removed.
'''

product_df=pd.read_excel("C:\\Users\\vakit\\amazon_data2.xlsx")
print(len(product_df))

# Renaming specific columns
product_df = product_df.rename(columns={'title': 'product_name', 'subtitle': 'product_company'})

# Removing rows where 'title' equals 'Okay?'
product_df = product_df[product_df['product_name'] != 'Okay?']

# Reset the index if needed
product_df.reset_index(drop=True, inplace=True)

14400


In [51]:
product_df

,ASIN,product_name,price,listPrice,imageUrl,detailPageURL,rating,totalReviews,product_company,isPrimeEligible,Category
0,B0C2NBW5LM,Yopela 14 Pack Natural Lip Balm in Bulk with ...,$5.59 ($0.40 / Count),NaN,https://m.media-amazon.com/images/I/51LlmDsUd8...,https://www.amazon.com/dp/B0C2NBW5LM,4.3,311.0,Yopela,1.0,lipbalm
1,B0C2NBW5LM,Yopela 14 Pack Natural Lip Balm in Bulk with ...,$5.59 ($0.40 / Count),NaN,https://m.media-amazon.com/images/I/51LlmDsUd8...,https://www.amazon.com/dp/B0C2NBW5LM,4.3,311.0,Yopela,1.0,lipbalm
2,B0C2NBW5LM,Yopela 14 Pack Natural Lip Balm in Bulk with ...,$5.59 ($0.40 / Count),NaN,https://m.media-amazon.com/images/I/51LlmDsUd8...,https://www.amazon.com/dp/B0C2NBW5LM,4.3,311.0,Yopela,1.0,lipbalm
3,B0C2NBW5LM,Yopela 14 Pack Natural Lip Balm in Bulk with ...,$5.59 ($0.40 / Count),NaN,https://m.media-amazon.com/images/I/51LlmDsUd8...,https://www.amazon.com/dp/B0C2NBW5LM,4.3,311.0,Yopela,1.0,lipbalm
4,B0C2NBW5LM,Yopela 14 Pack Natural Lip Balm in Bulk with ...,$5.59 ($0.40 / Count),NaN,https://m.media-amazon.com/images/I/51LlmDsUd8...,https://www.amazon.com/dp/B0C2NBW5LM,4.3,311.0,Yopela,1.0,lipbalm
...,...,...,...,...,...,...,...,...,...,...,...
14375,B0CGP293GL,𝟮𝟬𝟮𝟯 𝗨𝗽𝗴𝗿𝗮𝗱𝗲𝗱 Exfoliating Glove & Face Scrubbe...,$5.99 ($5.99 / Count),NaN,https://m.media-amazon.com/images/I/51imtvlk9z...,https://www.amazon.com/dp/B0CGP293GL,4.7,3573.0,NaN,1.0,exfoliator
14376,B0CGP293GL,𝟮𝟬𝟮𝟯 𝗨𝗽𝗴𝗿𝗮𝗱𝗲𝗱 Exfoliating Glove & Face Scrubbe...,$5.99 ($5.99 / Count),NaN,https://m.media-amazon.com/images/I/51imtvlk9z...,https://www.amazon.com/dp/B0CGP293GL,4.7,3573.0,NaN,1.0,exfoliator
14377,B0CGP293GL,𝟮𝟬𝟮𝟯 𝗨𝗽𝗴𝗿𝗮𝗱𝗲𝗱 Exfoliating Glove & Face Scrubbe...,$5.99 ($5.99 / Count),NaN,https://m.media-amazon.com/images/I/51imtvlk9z...,https://www.amazon.com/dp/B0CGP293GL,4.7,3573.0,NaN,1.0,exfoliator
14378,B0CGP293GL,𝟮𝟬𝟮𝟯 𝗨𝗽𝗴𝗿𝗮𝗱𝗲𝗱 Exfoliating Glove & Face Scrubbe...,$5.99 ($5.99 / Count),NaN,https://m.media-amazon.com/images/I/51imtvlk9z...,https://www.amazon.com/dp/B0CGP293GL,4.7,3573.0,NaN,1.0,exfoliator


In [52]:
'''
The resultant dataset reduced from 14400 rows to 14380 rows.

A closer inspection shows that this dataset also extracted a lot of makeup products like lipstick, lipgloss, 
mascara, concealer, foundation , eyeliner, fixing powder, concealer, lipgloss, eye tint, primer, eye shadow, etc. 
Our focus is only on skincare products so these makeup products were removed.'''

# Listing keywords to search for
excluding_keywords = [
    'lipstick', 'lipgloss', 'mascara', 'concealer', 'foundation',
    'eyeliner', 'fixing powder', 'concealer', 'lipgloss', 'eye tint',
    'primer', 'eye shadow'
]

# Filtering rows where 'product_name' contains any of the excluding keywords
removal_rows = product_df['product_name'].str.contains('|'.join(excluding_keywords), case=False, regex=True)

In [53]:
# Drop rows based on the removal_rows
product_df = product_df[~removal_rows]

# Reset the index if needed
product_df.reset_index(drop=True, inplace=True)

product_df

,ASIN,product_name,price,listPrice,imageUrl,detailPageURL,rating,totalReviews,product_company,isPrimeEligible,Category
0,B0C2NBW5LM,Yopela 14 Pack Natural Lip Balm in Bulk with ...,$5.59 ($0.40 / Count),NaN,https://m.media-amazon.com/images/I/51LlmDsUd8...,https://www.amazon.com/dp/B0C2NBW5LM,4.3,311.0,Yopela,1.0,lipbalm
1,B0C2NBW5LM,Yopela 14 Pack Natural Lip Balm in Bulk with ...,$5.59 ($0.40 / Count),NaN,https://m.media-amazon.com/images/I/51LlmDsUd8...,https://www.amazon.com/dp/B0C2NBW5LM,4.3,311.0,Yopela,1.0,lipbalm
2,B0C2NBW5LM,Yopela 14 Pack Natural Lip Balm in Bulk with ...,$5.59 ($0.40 / Count),NaN,https://m.media-amazon.com/images/I/51LlmDsUd8...,https://www.amazon.com/dp/B0C2NBW5LM,4.3,311.0,Yopela,1.0,lipbalm
3,B0C2NBW5LM,Yopela 14 Pack Natural Lip Balm in Bulk with ...,$5.59 ($0.40 / Count),NaN,https://m.media-amazon.com/images/I/51LlmDsUd8...,https://www.amazon.com/dp/B0C2NBW5LM,4.3,311.0,Yopela,1.0,lipbalm
4,B0C2NBW5LM,Yopela 14 Pack Natural Lip Balm in Bulk with ...,$5.59 ($0.40 / Count),NaN,https://m.media-amazon.com/images/I/51LlmDsUd8...,https://www.amazon.com/dp/B0C2NBW5LM,4.3,311.0,Yopela,1.0,lipbalm
...,...,...,...,...,...,...,...,...,...,...,...
14034,B0CGP293GL,𝟮𝟬𝟮𝟯 𝗨𝗽𝗴𝗿𝗮𝗱𝗲𝗱 Exfoliating Glove & Face Scrubbe...,$5.99 ($5.99 / Count),NaN,https://m.media-amazon.com/images/I/51imtvlk9z...,https://www.amazon.com/dp/B0CGP293GL,4.7,3573.0,NaN,1.0,exfoliator
14035,B0CGP293GL,𝟮𝟬𝟮𝟯 𝗨𝗽𝗴𝗿𝗮𝗱𝗲𝗱 Exfoliating Glove & Face Scrubbe...,$5.99 ($5.99 / Count),NaN,https://m.media-amazon.com/images/I/51imtvlk9z...,https://www.amazon.com/dp/B0CGP293GL,4.7,3573.0,NaN,1.0,exfoliator
14036,B0CGP293GL,𝟮𝟬𝟮𝟯 𝗨𝗽𝗴𝗿𝗮𝗱𝗲𝗱 Exfoliating Glove & Face Scrubbe...,$5.99 ($5.99 / Count),NaN,https://m.media-amazon.com/images/I/51imtvlk9z...,https://www.amazon.com/dp/B0CGP293GL,4.7,3573.0,NaN,1.0,exfoliator
14037,B0CGP293GL,𝟮𝟬𝟮𝟯 𝗨𝗽𝗴𝗿𝗮𝗱𝗲𝗱 Exfoliating Glove & Face Scrubbe...,$5.99 ($5.99 / Count),NaN,https://m.media-amazon.com/images/I/51imtvlk9z...,https://www.amazon.com/dp/B0CGP293GL,4.7,3573.0,NaN,1.0,exfoliator


In [54]:
'''
The resultant dataset further reduced to 14039 rows. We found that there are 12490 duplicates in the dataset. 
This means there are only 1482 unique products that have multiple occurrences across many pages. The keywords we have used 
also coincide with each other. This can lead to multi-fuctioning products being downloaded under different keywords. 
'''

#copying the dataframe 
product_df1= product_df.copy(deep=True)

#finding duplicates
duplicates = product_df1[product_df1.duplicated()]

#printing total no. of duplicates
total_duplicates = len(duplicates)
total_duplicates

12490

In [55]:
#dropping duplicates
product_df1.drop_duplicates(inplace=True)

#resetting index
product_df1 = product_df1.reset_index(drop=True)
product_df1

,ASIN,product_name,price,listPrice,imageUrl,detailPageURL,rating,totalReviews,product_company,isPrimeEligible,Category
0,B0C2NBW5LM,Yopela 14 Pack Natural Lip Balm in Bulk with ...,$5.59 ($0.40 / Count),NaN,https://m.media-amazon.com/images/I/51LlmDsUd8...,https://www.amazon.com/dp/B0C2NBW5LM,4.3,311.0,Yopela,1.0,lipbalm
1,B0B43WPH44,#OOTD Vegan Hydrating Korean Face Mask Sheet P...,$33.90 ($1.13 / Count),NaN,https://m.media-amazon.com/images/I/515Vy33-4b...,https://www.amazon.com/dp/B0B43WPH44,4.5,381.0,#OOTD,1.0,sheet mask
2,B08KDGFD1D,[20Packs] KF-94 - Face Protective Mask for Adu...,$17.88 ($0.89 / Count),$22.88 ($1.14 / Count),https://m.media-amazon.com/images/I/419d-rrYgS...,https://www.amazon.com/dp/B08KDGFD1D,4.5,11220.0,NaN,1.0,face mask
3,B083QCJPDD,[Beauty of Joseon] Centella Asiatica Calming M...,$15.52 ($15.37 / Fl Oz),$25.00 ($24.75 / Fl Oz),https://m.media-amazon.com/images/I/31YEAvTCvN...,https://www.amazon.com/dp/B083QCJPDD,4.5,135.0,Beauty of Joseon,1.0,sheet mask
4,B0B3R661JP,[Beauty of Joseon] Radiance Cleansing Balm (10...,$17.10 ($5.06 / Ounce),$19.00 ($5.62 / Ounce),https://m.media-amazon.com/images/I/21Pppxr2Fg...,https://www.amazon.com/dp/B0B3R661JP,4.6,302.0,Beauty of Joseon,1.0,cleansing oil
...,...,...,...,...,...,...,...,...,...,...,...
1544,B0894NFZNN,ZUBREX 50 Pcs Disposable 3 Ply Safety Face Mas...,$19.99 ($0.40 / Count),NaN,https://m.media-amazon.com/images/I/513InOwjk9...,https://www.amazon.com/dp/B0894NFZNN,4.7,1753.0,NaN,1.0,face mask
1545,B0150GNTQK,Zum Face Sugar Facial Scrub - Lemongrass - 4 oz,$15.25 ($3.81 / Ounce),NaN,https://m.media-amazon.com/images/I/41VQIx37pj...,https://www.amazon.com/dp/B0150GNTQK,4.5,166.0,Zum,1.0,face scrub
1546,B07P8ZN5JS,𝗧𝗢𝗣 𝟮𝟬𝟮𝟯* Hyaluronic Acid Face Serum - 2.5% Pu...,$9.99 ($9.99 / Fl Oz),$11.99 ($11.99 / Fl Oz),https://m.media-amazon.com/images/I/51nfiw3lEq...,https://www.amazon.com/dp/B07P8ZN5JS,4.3,24024.0,Eclat Skincare,1.0,serum
1547,B09MN49M76,"𝗪𝗜𝗡𝗡𝗘𝗥 𝟮𝟬𝟮𝟯* Turmeric Face Mask, Aztec Clay Ma...",$8.99 ($3.64 / Ounce),NaN,https://m.media-amazon.com/images/I/512TLxEv6z...,https://www.amazon.com/dp/B09MN49M76,4.5,198.0,Betty Rose's Botanicals,1.0,clay mask


In [56]:
'''
We do not need images of the products so let us drop them too. 
'''
product_df1.drop(columns=['imageUrl'], axis=1, inplace=True)
product_df1

# Reset the index if needed
product_df.reset_index(drop=True, inplace=True)

In [57]:
#product_df1.to_csv(r'product_df1.csv', index=False)

In [58]:
product_df1[product_df1['product_name'].str.startswith('essence')]

,ASIN,product_name,price,listPrice,detailPageURL,rating,totalReviews,product_company,isPrimeEligible,Category
566,B09KG5PLW1,essence | 3-Pack Extreme Shine Volume Lip Glos...,$11.97,NaN,https://www.amazon.com/dp/B09KG5PLW1,4.6,576.0,essence cosmetics,1.0,essence
567,B0B56TXR7W,essence | Baby Got Blush | Easy to Apply & Ble...,$4.99 ($27.72 / Fl Oz),NaN,https://www.amazon.com/dp/B0B56TXR7W,4.0,65.0,essence cosmetics,1.0,essence
568,B0BPDL2L95,essence | Baby Got Bronze | Cream Bronzer Stic...,$4.99 ($26.26 / Fl Oz),NaN,https://www.amazon.com/dp/B0BPDL2L95,4.0,318.0,essence cosmetics,1.0,essence
569,B09DHSBDQJ,essence | Baby Got Brow! Eyebrow Pencil | Long...,$4.99 ($4.99 / Count),NaN,https://www.amazon.com/dp/B09DHSBDQJ,4.4,437.0,essence cosmetics,1.0,essence
570,B076CS63W6,essence | Brighten Up! Banana Powder | Mattify...,$9.95 ($32.10 / Ounce),$13.50 ($43.55 / Ounce),https://www.amazon.com/dp/B076CS63W6,4.0,695.0,Essence,1.0,essence
571,B08YFFFW3M,"essence | Cranberry Lip Oil | pH-Reactive, Hyd...",$4.99 ($35.64 / Fl Oz),NaN,https://www.amazon.com/dp/B08YFFFW3M,4.4,55.0,essence cosmetics,1.0,essence
572,B09R83Z7PS,essence | Extreme Care Hydrating Glossy Lip Ba...,$4.99 ($4.99 / Count),NaN,https://www.amazon.com/dp/B09R83Z7PS,3.9,82.0,essence cosmetics,1.0,essence
573,B09VH8431N,essence | Fix It Like A Boss Transparent Brow ...,$4.99 ($17.21 / Fl Oz),NaN,https://www.amazon.com/dp/B09VH8431N,4.0,201.0,essence cosmetics,1.0,essence
574,B0C9SM8SRM,essence | Heart Core Fruity Lip Balm Set | 4 L...,$10.99 ($26.17 / Ounce),NaN,https://www.amazon.com/dp/B0C9SM8SRM,1.0,1.0,essence cosmetics,1.0,essence
575,B08XZVRGNR,"essence | HELLO, GOOD STUFF! Milky Moisturizer...",$8.98 ($8.98 / Fl Oz),NaN,https://www.amazon.com/dp/B08XZVRGNR,4.4,111.0,essence cosmetics,1.0,essence


In [59]:
'''
Upon closer inspection, we notice that titles that start with "essence" have a lot of makeup products disguised as skincare.
These need to be removed. Different products have different names and all of these might not be identifyable on code. 
Hence, this is done by manually identifying the items and creating a list of their corresponding row indices. 
Subsequently, the dataframe rows corresponding to the list elements are dropped. In addition to this, all products that 
start with "Essence" are not skincare products so they were dropped too.
'''

product_df1[product_df1['product_name'].str.startswith('essence')]
indices_to_remove=[28,566, 567, 568, 569, 570,573, 576, 577, 578, 579, 580]

# Remove the rows with the specified indices
product_df1 = product_df1.drop(indices_to_remove)

In [60]:
#removing all products whose product_name is starting with "Essence"
product_df1=product_df1[~product_df1['product_name'].str.startswith('Essence')]

# Reset the index if needed
product_df.reset_index(drop=True, inplace=True)

In [61]:
#collecting all rows whose product_company is na
na_rows = product_df1[product_df1['product_company'].isna()]
len(na_rows)

'''
There are 356 products with unknown companies. Sometimes product_names can be written in a different format which makes it
hard to determine the company name. It is important to fill in the company names during this scenario. Other times, the 
product really does not have a company name. We are going to find this out. 
'''

#converting the na_rows into a dataframe
na_rows_df = pd.DataFrame(na_rows)
na_rows_df

'''
This dataset also has disposable face masks. Let us start with products that have disposable in their product_name
'''
#creating face mask related keywords that will be used to eliminate rows
face_mask_keywords=['Disposable', 'KF-94','KF94', 'K-94', 'cloth', 'scrubber', 'Weighted', 'KN95', 'Plastic', 'book']

# Filtering rows where 'product_name' contains any of the face_mask_keywords
face_mask_rows = product_df1['product_name'].str.contains('|'.join(face_mask_keywords), case=False, regex=True)
product_df1=product_df1[~face_mask_rows]

In [62]:
'''
28 products tagged sleep mask may or may not be skincare products. The ones which are not must be removed.
'''
#creating sleep mask related keywords that will be used to eliminate rows
sleep_mask=['sleep mask', 'Sleep mask', 'Sleep Mask', 'sleep Mask']

# Filtering rows where 'product_name' contains any of the sleep_mask keywords
sleep_mask_rows = product_df1['product_name'].str.contains('|'.join(sleep_mask), case=False, regex=True)
product_df1=product_df1[~sleep_mask_rows]

In [63]:
#making a list of indices that have non-skincare products through manual observation
non_skincare=[243, 244, 774, 1340, 37, 20]
#dropping the non-skincare related rows
product_df1=product_df1.drop(non_skincare)

#resetting index
product_df1 = product_df1.reset_index(drop=True)
product_df1

,ASIN,product_name,price,listPrice,detailPageURL,rating,totalReviews,product_company,isPrimeEligible,Category
0,B0C2NBW5LM,Yopela 14 Pack Natural Lip Balm in Bulk with ...,$5.59 ($0.40 / Count),NaN,https://www.amazon.com/dp/B0C2NBW5LM,4.3,311.0,Yopela,1.0,lipbalm
1,B0B43WPH44,#OOTD Vegan Hydrating Korean Face Mask Sheet P...,$33.90 ($1.13 / Count),NaN,https://www.amazon.com/dp/B0B43WPH44,4.5,381.0,#OOTD,1.0,sheet mask
2,B083QCJPDD,[Beauty of Joseon] Centella Asiatica Calming M...,$15.52 ($15.37 / Fl Oz),$25.00 ($24.75 / Fl Oz),https://www.amazon.com/dp/B083QCJPDD,4.5,135.0,Beauty of Joseon,1.0,sheet mask
3,B0B3R661JP,[Beauty of Joseon] Radiance Cleansing Balm (10...,$17.10 ($5.06 / Ounce),$19.00 ($5.62 / Ounce),https://www.amazon.com/dp/B0B3R661JP,4.6,302.0,Beauty of Joseon,1.0,cleansing oil
4,B0B3R661JP,[Beauty of Joseon] Radiance Cleansing Balm (10...,$17.10 ($5.06 / Ounce),$19.00 ($5.62 / Ounce),https://www.amazon.com/dp/B0B3R661JP,4.6,302.0,Beauty of Joseon,1.0,cleansing balm
...,...,...,...,...,...,...,...,...,...,...
1350,B0BQ3R8LH7,ZeoClay Face Mask - 100% Organic Spa Quality -...,$38.24 ($7.23 / Ounce),$44.99 ($8.50 / Ounce),https://www.amazon.com/dp/B0BQ3R8LH7,5.0,20.0,ZeoClay,1.0,clay mask
1351,B07S472V7D,ZO Skin Health Gentle Cleanser 6.7 Fl. Oz.,$36.90 ($5.51 / Fl Oz),$40.97 ($6.11 / Fl Oz),https://www.amazon.com/dp/B07S472V7D,4.7,606.0,ZO SKIN HEALTH,1.0,cleanser
1352,B0150GNTQK,Zum Face Sugar Facial Scrub - Lemongrass - 4 oz,$15.25 ($3.81 / Ounce),NaN,https://www.amazon.com/dp/B0150GNTQK,4.5,166.0,Zum,1.0,face scrub
1353,B07P8ZN5JS,𝗧𝗢𝗣 𝟮𝟬𝟮𝟯* Hyaluronic Acid Face Serum - 2.5% Pu...,$9.99 ($9.99 / Fl Oz),$11.99 ($11.99 / Fl Oz),https://www.amazon.com/dp/B07P8ZN5JS,4.3,24024.0,Eclat Skincare,1.0,serum


In [64]:
'''Similarily, products tagged recipe may or may not have skincare products in them.'''

#checking for products that have recipe keyword in them 
recipe_words=['recipe', 'Recipe']

#checking for products that have recipe keyword in them
recipe_word_rows = product_df1['product_name'].str.contains('|'.join(recipe_words), case=False, regex=True)

#only rows 650, 1161 have non-skincare related products
non_recipe_words=[650, 1161]
product_df1=product_df1.drop(non_recipe_words)

In [65]:
'''
This process ensures that the data remains organized and retains its integrity, preventing the loss of important 
information or progress during data manipulation. It also helps avoid misalignment.'''

#resetting index
product_df1 = product_df1.reset_index(drop=True)
product_df1

,ASIN,product_name,price,listPrice,detailPageURL,rating,totalReviews,product_company,isPrimeEligible,Category
0,B0C2NBW5LM,Yopela 14 Pack Natural Lip Balm in Bulk with ...,$5.59 ($0.40 / Count),NaN,https://www.amazon.com/dp/B0C2NBW5LM,4.3,311.0,Yopela,1.0,lipbalm
1,B0B43WPH44,#OOTD Vegan Hydrating Korean Face Mask Sheet P...,$33.90 ($1.13 / Count),NaN,https://www.amazon.com/dp/B0B43WPH44,4.5,381.0,#OOTD,1.0,sheet mask
2,B083QCJPDD,[Beauty of Joseon] Centella Asiatica Calming M...,$15.52 ($15.37 / Fl Oz),$25.00 ($24.75 / Fl Oz),https://www.amazon.com/dp/B083QCJPDD,4.5,135.0,Beauty of Joseon,1.0,sheet mask
3,B0B3R661JP,[Beauty of Joseon] Radiance Cleansing Balm (10...,$17.10 ($5.06 / Ounce),$19.00 ($5.62 / Ounce),https://www.amazon.com/dp/B0B3R661JP,4.6,302.0,Beauty of Joseon,1.0,cleansing oil
4,B0B3R661JP,[Beauty of Joseon] Radiance Cleansing Balm (10...,$17.10 ($5.06 / Ounce),$19.00 ($5.62 / Ounce),https://www.amazon.com/dp/B0B3R661JP,4.6,302.0,Beauty of Joseon,1.0,cleansing balm
...,...,...,...,...,...,...,...,...,...,...
1348,B0BQ3R8LH7,ZeoClay Face Mask - 100% Organic Spa Quality -...,$38.24 ($7.23 / Ounce),$44.99 ($8.50 / Ounce),https://www.amazon.com/dp/B0BQ3R8LH7,5.0,20.0,ZeoClay,1.0,clay mask
1349,B07S472V7D,ZO Skin Health Gentle Cleanser 6.7 Fl. Oz.,$36.90 ($5.51 / Fl Oz),$40.97 ($6.11 / Fl Oz),https://www.amazon.com/dp/B07S472V7D,4.7,606.0,ZO SKIN HEALTH,1.0,cleanser
1350,B0150GNTQK,Zum Face Sugar Facial Scrub - Lemongrass - 4 oz,$15.25 ($3.81 / Ounce),NaN,https://www.amazon.com/dp/B0150GNTQK,4.5,166.0,Zum,1.0,face scrub
1351,B07P8ZN5JS,𝗧𝗢𝗣 𝟮𝟬𝟮𝟯* Hyaluronic Acid Face Serum - 2.5% Pu...,$9.99 ($9.99 / Fl Oz),$11.99 ($11.99 / Fl Oz),https://www.amazon.com/dp/B07P8ZN5JS,4.3,24024.0,Eclat Skincare,1.0,serum


In [66]:
'''Similarily, products tagged "book" may or may not have skincare products in them. 
'''
#creating keywords to filter by 
book_words=['Book', 'book']
#filtering rows related to those keywords
Book_instances=product_df1['product_name'].str.contains('|'.join(book_words), case=False, regex=True)
#removing those rows
product_df1=product_df1[~Book_instances]
#displaying result
product_df1

,ASIN,product_name,price,listPrice,detailPageURL,rating,totalReviews,product_company,isPrimeEligible,Category
0,B0C2NBW5LM,Yopela 14 Pack Natural Lip Balm in Bulk with ...,$5.59 ($0.40 / Count),NaN,https://www.amazon.com/dp/B0C2NBW5LM,4.3,311.0,Yopela,1.0,lipbalm
1,B0B43WPH44,#OOTD Vegan Hydrating Korean Face Mask Sheet P...,$33.90 ($1.13 / Count),NaN,https://www.amazon.com/dp/B0B43WPH44,4.5,381.0,#OOTD,1.0,sheet mask
2,B083QCJPDD,[Beauty of Joseon] Centella Asiatica Calming M...,$15.52 ($15.37 / Fl Oz),$25.00 ($24.75 / Fl Oz),https://www.amazon.com/dp/B083QCJPDD,4.5,135.0,Beauty of Joseon,1.0,sheet mask
3,B0B3R661JP,[Beauty of Joseon] Radiance Cleansing Balm (10...,$17.10 ($5.06 / Ounce),$19.00 ($5.62 / Ounce),https://www.amazon.com/dp/B0B3R661JP,4.6,302.0,Beauty of Joseon,1.0,cleansing oil
4,B0B3R661JP,[Beauty of Joseon] Radiance Cleansing Balm (10...,$17.10 ($5.06 / Ounce),$19.00 ($5.62 / Ounce),https://www.amazon.com/dp/B0B3R661JP,4.6,302.0,Beauty of Joseon,1.0,cleansing balm
...,...,...,...,...,...,...,...,...,...,...
1348,B0BQ3R8LH7,ZeoClay Face Mask - 100% Organic Spa Quality -...,$38.24 ($7.23 / Ounce),$44.99 ($8.50 / Ounce),https://www.amazon.com/dp/B0BQ3R8LH7,5.0,20.0,ZeoClay,1.0,clay mask
1349,B07S472V7D,ZO Skin Health Gentle Cleanser 6.7 Fl. Oz.,$36.90 ($5.51 / Fl Oz),$40.97 ($6.11 / Fl Oz),https://www.amazon.com/dp/B07S472V7D,4.7,606.0,ZO SKIN HEALTH,1.0,cleanser
1350,B0150GNTQK,Zum Face Sugar Facial Scrub - Lemongrass - 4 oz,$15.25 ($3.81 / Ounce),NaN,https://www.amazon.com/dp/B0150GNTQK,4.5,166.0,Zum,1.0,face scrub
1351,B07P8ZN5JS,𝗧𝗢𝗣 𝟮𝟬𝟮𝟯* Hyaluronic Acid Face Serum - 2.5% Pu...,$9.99 ($9.99 / Fl Oz),$11.99 ($11.99 / Fl Oz),https://www.amazon.com/dp/B07P8ZN5JS,4.3,24024.0,Eclat Skincare,1.0,serum


In [67]:
'''
There is a movie called 8 seconds so let us remove it too.
'''
#finding 8seconds row
annoying=product_df1['product_name'].str.contains('8 Seconds', case=False, regex=True)
#removing 8 Seconds
product_df1=product_df1[~annoying]

#resetting index
product_df1 = product_df1.reset_index(drop=True)
product_df1

,ASIN,product_name,price,listPrice,detailPageURL,rating,totalReviews,product_company,isPrimeEligible,Category
0,B0C2NBW5LM,Yopela 14 Pack Natural Lip Balm in Bulk with ...,$5.59 ($0.40 / Count),NaN,https://www.amazon.com/dp/B0C2NBW5LM,4.3,311.0,Yopela,1.0,lipbalm
1,B0B43WPH44,#OOTD Vegan Hydrating Korean Face Mask Sheet P...,$33.90 ($1.13 / Count),NaN,https://www.amazon.com/dp/B0B43WPH44,4.5,381.0,#OOTD,1.0,sheet mask
2,B083QCJPDD,[Beauty of Joseon] Centella Asiatica Calming M...,$15.52 ($15.37 / Fl Oz),$25.00 ($24.75 / Fl Oz),https://www.amazon.com/dp/B083QCJPDD,4.5,135.0,Beauty of Joseon,1.0,sheet mask
3,B0B3R661JP,[Beauty of Joseon] Radiance Cleansing Balm (10...,$17.10 ($5.06 / Ounce),$19.00 ($5.62 / Ounce),https://www.amazon.com/dp/B0B3R661JP,4.6,302.0,Beauty of Joseon,1.0,cleansing oil
4,B0B3R661JP,[Beauty of Joseon] Radiance Cleansing Balm (10...,$17.10 ($5.06 / Ounce),$19.00 ($5.62 / Ounce),https://www.amazon.com/dp/B0B3R661JP,4.6,302.0,Beauty of Joseon,1.0,cleansing balm
...,...,...,...,...,...,...,...,...,...,...
1346,B0BQ3R8LH7,ZeoClay Face Mask - 100% Organic Spa Quality -...,$38.24 ($7.23 / Ounce),$44.99 ($8.50 / Ounce),https://www.amazon.com/dp/B0BQ3R8LH7,5.0,20.0,ZeoClay,1.0,clay mask
1347,B07S472V7D,ZO Skin Health Gentle Cleanser 6.7 Fl. Oz.,$36.90 ($5.51 / Fl Oz),$40.97 ($6.11 / Fl Oz),https://www.amazon.com/dp/B07S472V7D,4.7,606.0,ZO SKIN HEALTH,1.0,cleanser
1348,B0150GNTQK,Zum Face Sugar Facial Scrub - Lemongrass - 4 oz,$15.25 ($3.81 / Ounce),NaN,https://www.amazon.com/dp/B0150GNTQK,4.5,166.0,Zum,1.0,face scrub
1349,B07P8ZN5JS,𝗧𝗢𝗣 𝟮𝟬𝟮𝟯* Hyaluronic Acid Face Serum - 2.5% Pu...,$9.99 ($9.99 / Fl Oz),$11.99 ($11.99 / Fl Oz),https://www.amazon.com/dp/B07P8ZN5JS,4.3,24024.0,Eclat Skincare,1.0,serum


In [68]:
'''
On manual observation, 31 unrelevant products like physical eye masks, books on skincare etc. were found again. 
Their respective product ids from the 'ASIN' column were copied into unwanted_id list. The rows corresponding to these 31 ids
were dropped.
'''

unwanted_id=['B00S7HYEKK', 'B07HBQ61TS', 'B0B63WB7J3', 'B078S5YT6R', 'B07MYQNV6Z', 'B08GTQH5GG', 'B0725XFZCP', 
             'B083TYGF38', 'B071JC2CGM', 'B00P52A3YQ', 'B016RFJDL8', '1983490016', 'B085466ZPZ', 'B08M4CS1J5', 
             'B07FY24VWZ', 'B081W68DP2', 'B006R962UA', 'B00000IG3O', 'B097G9BS19', 'B071KQ691V', 'B07YN814J6', 
             'B08DBNH57P', 'B003ODHO8Q', 'B01B24A5O8', 'B08X65PG5S', 'B00RRT31M6', '1620553279', 'B001JJH8HY', 
             'B0C9S86QWF', 'B0BY3R55FD', 'B07W5MT57S', 'B00HWBUBE', 'B08DJH5ZRF','B088GNK5WH', 'B00KO8685Y', 
             'B09V22N8GX', 'B0CDC29V9M', 'B0CDCKKXHF', 'B08YRZ6G1N','B08YRXDCHB', 'B00II5RSEU', 'B084YQJ7VS',
             'B074TBYWGS', 'B06XD4TPWP', 'B08JCK6DWC', 'B010WC5UNW', 'B0CC5GH128', 'B00917HHWC', 'B00FEBARJE',
             'B07W6XCBHG', 'B086ZVFX1R']


#filtering rows related to those ASIN values
unwanted_instances=product_df1['ASIN'].str.contains('|'.join(unwanted_id), case=False, regex=True)
product_df1=product_df1[~unwanted_instances]

#resetting index
product_df1 = product_df1.reset_index(drop=True)
product_df1

,ASIN,product_name,price,listPrice,detailPageURL,rating,totalReviews,product_company,isPrimeEligible,Category
0,B0C2NBW5LM,Yopela 14 Pack Natural Lip Balm in Bulk with ...,$5.59 ($0.40 / Count),NaN,https://www.amazon.com/dp/B0C2NBW5LM,4.3,311.0,Yopela,1.0,lipbalm
1,B0B43WPH44,#OOTD Vegan Hydrating Korean Face Mask Sheet P...,$33.90 ($1.13 / Count),NaN,https://www.amazon.com/dp/B0B43WPH44,4.5,381.0,#OOTD,1.0,sheet mask
2,B083QCJPDD,[Beauty of Joseon] Centella Asiatica Calming M...,$15.52 ($15.37 / Fl Oz),$25.00 ($24.75 / Fl Oz),https://www.amazon.com/dp/B083QCJPDD,4.5,135.0,Beauty of Joseon,1.0,sheet mask
3,B0B3R661JP,[Beauty of Joseon] Radiance Cleansing Balm (10...,$17.10 ($5.06 / Ounce),$19.00 ($5.62 / Ounce),https://www.amazon.com/dp/B0B3R661JP,4.6,302.0,Beauty of Joseon,1.0,cleansing oil
4,B0B3R661JP,[Beauty of Joseon] Radiance Cleansing Balm (10...,$17.10 ($5.06 / Ounce),$19.00 ($5.62 / Ounce),https://www.amazon.com/dp/B0B3R661JP,4.6,302.0,Beauty of Joseon,1.0,cleansing balm
...,...,...,...,...,...,...,...,...,...,...
1297,B0BQ3R8LH7,ZeoClay Face Mask - 100% Organic Spa Quality -...,$38.24 ($7.23 / Ounce),$44.99 ($8.50 / Ounce),https://www.amazon.com/dp/B0BQ3R8LH7,5.0,20.0,ZeoClay,1.0,clay mask
1298,B07S472V7D,ZO Skin Health Gentle Cleanser 6.7 Fl. Oz.,$36.90 ($5.51 / Fl Oz),$40.97 ($6.11 / Fl Oz),https://www.amazon.com/dp/B07S472V7D,4.7,606.0,ZO SKIN HEALTH,1.0,cleanser
1299,B0150GNTQK,Zum Face Sugar Facial Scrub - Lemongrass - 4 oz,$15.25 ($3.81 / Ounce),NaN,https://www.amazon.com/dp/B0150GNTQK,4.5,166.0,Zum,1.0,face scrub
1300,B07P8ZN5JS,𝗧𝗢𝗣 𝟮𝟬𝟮𝟯* Hyaluronic Acid Face Serum - 2.5% Pu...,$9.99 ($9.99 / Fl Oz),$11.99 ($11.99 / Fl Oz),https://www.amazon.com/dp/B07P8ZN5JS,4.3,24024.0,Eclat Skincare,1.0,serum


In [69]:
#final_semi_clean=product_df1
#final_semi_clean.to_csv(r'final_semi_clean.csv', index=False)

In [70]:
'''
Check for and remove duplicate product_names.
'''

# Check for duplicates in the 'product_name' column
duplicate_mask = product_df1.duplicated(subset='product_name', keep='first')
# Remove duplicate rows based on the mask
product_df1 = product_df1[~duplicate_mask]
# Display the DataFrame without duplicates
product_df1

,ASIN,product_name,price,listPrice,detailPageURL,rating,totalReviews,product_company,isPrimeEligible,Category
0,B0C2NBW5LM,Yopela 14 Pack Natural Lip Balm in Bulk with ...,$5.59 ($0.40 / Count),NaN,https://www.amazon.com/dp/B0C2NBW5LM,4.3,311.0,Yopela,1.0,lipbalm
1,B0B43WPH44,#OOTD Vegan Hydrating Korean Face Mask Sheet P...,$33.90 ($1.13 / Count),NaN,https://www.amazon.com/dp/B0B43WPH44,4.5,381.0,#OOTD,1.0,sheet mask
2,B083QCJPDD,[Beauty of Joseon] Centella Asiatica Calming M...,$15.52 ($15.37 / Fl Oz),$25.00 ($24.75 / Fl Oz),https://www.amazon.com/dp/B083QCJPDD,4.5,135.0,Beauty of Joseon,1.0,sheet mask
3,B0B3R661JP,[Beauty of Joseon] Radiance Cleansing Balm (10...,$17.10 ($5.06 / Ounce),$19.00 ($5.62 / Ounce),https://www.amazon.com/dp/B0B3R661JP,4.6,302.0,Beauty of Joseon,1.0,cleansing oil
5,B00OZ6W8DW,[Cosrx] Salicylic Acid Daily Gentle Cleanser 1...,$13.99 ($2.76 / Fl Oz),NaN,https://www.amazon.com/dp/B00OZ6W8DW,4.6,1581.0,COSRX,1.0,cleanser
...,...,...,...,...,...,...,...,...,...,...
1297,B0BQ3R8LH7,ZeoClay Face Mask - 100% Organic Spa Quality -...,$38.24 ($7.23 / Ounce),$44.99 ($8.50 / Ounce),https://www.amazon.com/dp/B0BQ3R8LH7,5.0,20.0,ZeoClay,1.0,clay mask
1298,B07S472V7D,ZO Skin Health Gentle Cleanser 6.7 Fl. Oz.,$36.90 ($5.51 / Fl Oz),$40.97 ($6.11 / Fl Oz),https://www.amazon.com/dp/B07S472V7D,4.7,606.0,ZO SKIN HEALTH,1.0,cleanser
1299,B0150GNTQK,Zum Face Sugar Facial Scrub - Lemongrass - 4 oz,$15.25 ($3.81 / Ounce),NaN,https://www.amazon.com/dp/B0150GNTQK,4.5,166.0,Zum,1.0,face scrub
1300,B07P8ZN5JS,𝗧𝗢𝗣 𝟮𝟬𝟮𝟯* Hyaluronic Acid Face Serum - 2.5% Pu...,$9.99 ($9.99 / Fl Oz),$11.99 ($11.99 / Fl Oz),https://www.amazon.com/dp/B07P8ZN5JS,4.3,24024.0,Eclat Skincare,1.0,serum


In [71]:
"""
There are about 11 items in the dataset whose product name starts with the special character '[' or ']'. We must eliminate these. 
"""
product_df1[product_df1['product_name'].str.startswith('[')]

# Replacing '[' and ']' with nothing in the 'product_name' column
product_df1['product_name'] = product_df1['product_name'].str.replace(r'[\[\]]', '', regex=True)
product_df1

C:\Users\vakit\AppData\Local\Temp\ipykernel_20428\3880912128.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_df1['product_name'] = product_df1['product_name'].str.replace(r'[\[\]]', '', regex=True)


,ASIN,product_name,price,listPrice,detailPageURL,rating,totalReviews,product_company,isPrimeEligible,Category
0,B0C2NBW5LM,Yopela 14 Pack Natural Lip Balm in Bulk with ...,$5.59 ($0.40 / Count),NaN,https://www.amazon.com/dp/B0C2NBW5LM,4.3,311.0,Yopela,1.0,lipbalm
1,B0B43WPH44,#OOTD Vegan Hydrating Korean Face Mask Sheet P...,$33.90 ($1.13 / Count),NaN,https://www.amazon.com/dp/B0B43WPH44,4.5,381.0,#OOTD,1.0,sheet mask
2,B083QCJPDD,Beauty of Joseon Centella Asiatica Calming Mas...,$15.52 ($15.37 / Fl Oz),$25.00 ($24.75 / Fl Oz),https://www.amazon.com/dp/B083QCJPDD,4.5,135.0,Beauty of Joseon,1.0,sheet mask
3,B0B3R661JP,Beauty of Joseon Radiance Cleansing Balm (100m...,$17.10 ($5.06 / Ounce),$19.00 ($5.62 / Ounce),https://www.amazon.com/dp/B0B3R661JP,4.6,302.0,Beauty of Joseon,1.0,cleansing oil
5,B00OZ6W8DW,Cosrx Salicylic Acid Daily Gentle Cleanser 150...,$13.99 ($2.76 / Fl Oz),NaN,https://www.amazon.com/dp/B00OZ6W8DW,4.6,1581.0,COSRX,1.0,cleanser
...,...,...,...,...,...,...,...,...,...,...
1297,B0BQ3R8LH7,ZeoClay Face Mask - 100% Organic Spa Quality -...,$38.24 ($7.23 / Ounce),$44.99 ($8.50 / Ounce),https://www.amazon.com/dp/B0BQ3R8LH7,5.0,20.0,ZeoClay,1.0,clay mask
1298,B07S472V7D,ZO Skin Health Gentle Cleanser 6.7 Fl. Oz.,$36.90 ($5.51 / Fl Oz),$40.97 ($6.11 / Fl Oz),https://www.amazon.com/dp/B07S472V7D,4.7,606.0,ZO SKIN HEALTH,1.0,cleanser
1299,B0150GNTQK,Zum Face Sugar Facial Scrub - Lemongrass - 4 oz,$15.25 ($3.81 / Ounce),NaN,https://www.amazon.com/dp/B0150GNTQK,4.5,166.0,Zum,1.0,face scrub
1300,B07P8ZN5JS,𝗧𝗢𝗣 𝟮𝟬𝟮𝟯* Hyaluronic Acid Face Serum - 2.5% Pu...,$9.99 ($9.99 / Fl Oz),$11.99 ($11.99 / Fl Oz),https://www.amazon.com/dp/B07P8ZN5JS,4.3,24024.0,Eclat Skincare,1.0,serum


In [25]:
'''
Many popular companies like "Peter Thomas Roth", "I'm From", "TonyMoly", 'Mario Badescu' and "Beauty of Joseon" have 'na' in their product_company sections 
due to being formatted with special characters (e.g. [I'm From], [Cosrx], [Beauty of Joseon] etc. ). These must be 
corrected. This can be done through a combination of manual observation and coding. We can make a list of product names 
that have been formatted in such way and incorporate a for loop that finds the instance of the company name in the title and
enters the same company name in the relevant product_company column.  


#generating list of comay name strings through manual observation
company_names=['beauty of joseon', 'tonymoly', "i'm from", 'vichy mineral 89', 'uriage', 'babo botanicals', 'alpha-h', 
              'babor', 'beekman', 'belif', 'bgs', 'bioderma', 'blistex', 'bondi sands', 'brownmed', 'cailyn', 'caudalie',
             'cerave', 'chasin’ rabbits', 'derma e', 'derma-e', 'dermalogica', 'dermalogy by', 'dove', "dr. dan's", 
             'dr.ceuracle vegan', 'eau thermale', 'elemis', 'eltamd', 'eos', 'eve lom', 'image skincare', 'innisfree',
             'is clinical', 'it cosmetics', 'jack black', 'kao megurism', 'korres', 'la roche-posay', 'lancôme', 
             'laneige', "l'Occitane", 'mario badescu', 'mavogel', 'mediviz', 'missha', 'murad', 'neocutis', 'no-ad', 
              'now solutions', 'obagi', "o'keeffe's", 'oklahoma', 'olay', 'optase', 'ostrich', 'patchology', 'perricone', 
             'peter thomas roth', 'philosophy purity', 'shiseido', 'skinmedica', 'st ives', 'strivectin', 'sulwhasoo', 
             'sunday riley', 'tatcha', 'the better skin', 'the face shop' ]

# Defining the function to update the company based on the product name
def update_company(name):
    # Convert the product name to lowercase for case-insensitive comparison
    name_lower = name.lower()
    # Iterating through the list of company names and check if any of them is present in name_lower
    for company_name in company_names:
        print(f"Comparing: {name_lower} and {company_name}")
        if company_name in name_lower:
            return company_name.title()

    # If no match is found, returning 'na'
    return 'na'

# Iterating through the DataFrame rows and apply the update_company function
for index, row in product_df1.iterrows():
    product_name = row['product_name']
    final_company = update_company(product_name)  # Getting the final company name
    product_df1.at[index, 'company_name'] = final_company  # Assigning it to the 'company_name' column for the current row
'''

'\nMany popular companies like "Peter Thomas Roth", "I\'m From", "TonyMoly", \'Mario Badescu\' and "Beauty of Joseon" have \'na\' in their product_company sections \ndue to being formatted with special characters (e.g. [I\'m From], [Cosrx], [Beauty of Joseon] etc. ). These must be \ncorrected. This can be done through a combination of manual observation and coding. We can make a list of product names \nthat have been formatted in such way and incorporate a for loop that finds the instance of the company name in the title and\nenters the same company name in the relevant product_company column.  \n\n\n#generating list of comay name strings through manual observation\ncompany_names=[\'beauty of joseon\', \'tonymoly\', "i\'m from", \'vichy mineral 89\', \'uriage\', \'babo botanicals\', \'alpha-h\', \n              \'babor\', \'beekman\', \'belif\', \'bgs\', \'bioderma\', \'blistex\', \'bondi sands\', \'brownmed\', \'cailyn\', \'caudalie\',\n             \'cerave\', \'chasin’ rabbits\', \'

In [72]:
# Convert columns to lowercase
product_df1['product_name'] = product_df1['product_name'].str.lower()
product_df1['product_company'] = product_df1['product_company'].str.lower()


C:\Users\vakit\AppData\Local\Temp\ipykernel_20428\4190535299.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_df1['product_name'] = product_df1['product_name'].str.lower()
C:\Users\vakit\AppData\Local\Temp\ipykernel_20428\4190535299.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_df1['product_company'] = product_df1['product_company'].str.lower()


In [73]:
# Count NaN values before update
print("NaN count before update:", product_df1['product_company'].isna().sum())

NaN count before update: 188


In [74]:
product_df1['product_name'].apply(lambda x: ' '.join(x.split(' ')[0:4]))

0             yopela 14 pack natural
1       #ootd vegan hydrating korean
2          beauty of joseon centella
3          beauty of joseon radiance
5         cosrx salicylic acid daily
                    ...             
1297             zeoclay face mask -
1298           zo skin health gentle
1299           zum face sugar facial
1300       𝗧𝗢𝗣 𝟮𝟬𝟮𝟯* hyaluronic acid
1301      𝗪𝗜𝗡𝗡𝗘𝗥 𝟮𝟬𝟮𝟯* turmeric face
Name: product_name, Length: 1238, dtype: object

In [75]:
mask = product_df1['product_company'].isna()
product_df1.loc[mask, 'product_company'] = product_df1['product_name'].apply(lambda x: ' '.join(x.split(' ')[0:4]))

print("NaN count after update:", product_df1['product_company'].isna().sum())

NaN count after update: 0


In [77]:
#semi_clean=product_df1
#semi_clean.to_csv(r'semi_clean.csv', index=False)

In [81]:
'''
TYPE CONVERSION:
The price and listPrice columns in the dataset show instances like ($5.62 / Ounce). We must convert this to numeric data 
and remove instances like ($5.62 / Ounce). We must also convert the column datatypes.
'''
product_df1['listPrice'] = product_df1['listPrice'].astype('str').apply(lambda x: re.sub(r'\([^)]*\)', '', x))
product_df1['price'] = product_df1['price'].astype('str').apply(lambda x: re.sub(r'\([^)]*\)', '', x))

product_df1['listPrice'] = product_df1['listPrice'].apply(lambda x: x.replace('$','')).astype('float64')
product_df1['price'] = product_df1['price'].apply(lambda x: x.replace('$','')).astype('float64')

product_df1

C:\Users\vakit\AppData\Local\Temp\ipykernel_20428\746486014.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_df1['listPrice'] = product_df1['listPrice'].astype('str').apply(lambda x: re.sub(r'\([^)]*\)', '', x))
C:\Users\vakit\AppData\Local\Temp\ipykernel_20428\746486014.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_df1['price'] = product_df1['price'].astype('str').apply(lambda x: re.sub(r'\([^)]*\)', '', x))
C:\Users\vakit\AppData\Local\Temp\ipykernel_20428\746486014.py:4: SettingW

,ASIN,product_name,price,listPrice,detailPageURL,rating,totalReviews,product_company,isPrimeEligible,Category
0,B0C2NBW5LM,yopela 14 pack natural lip balm in bulk with ...,5.59,NaN,https://www.amazon.com/dp/B0C2NBW5LM,4.3,311.0,yopela,1.0,lipbalm
1,B0B43WPH44,#ootd vegan hydrating korean face mask sheet p...,33.90,NaN,https://www.amazon.com/dp/B0B43WPH44,4.5,381.0,#ootd,1.0,sheet mask
2,B083QCJPDD,beauty of joseon centella asiatica calming mas...,15.52,25.00,https://www.amazon.com/dp/B083QCJPDD,4.5,135.0,beauty of joseon,1.0,sheet mask
3,B0B3R661JP,beauty of joseon radiance cleansing balm (100m...,17.10,19.00,https://www.amazon.com/dp/B0B3R661JP,4.6,302.0,beauty of joseon,1.0,cleansing oil
5,B00OZ6W8DW,cosrx salicylic acid daily gentle cleanser 150...,13.99,NaN,https://www.amazon.com/dp/B00OZ6W8DW,4.6,1581.0,cosrx,1.0,cleanser
...,...,...,...,...,...,...,...,...,...,...
1297,B0BQ3R8LH7,zeoclay face mask - 100% organic spa quality -...,38.24,44.99,https://www.amazon.com/dp/B0BQ3R8LH7,5.0,20.0,zeoclay,1.0,clay mask
1298,B07S472V7D,zo skin health gentle cleanser 6.7 fl. oz.,36.90,40.97,https://www.amazon.com/dp/B07S472V7D,4.7,606.0,zo skin health,1.0,cleanser
1299,B0150GNTQK,zum face sugar facial scrub - lemongrass - 4 oz,15.25,NaN,https://www.amazon.com/dp/B0150GNTQK,4.5,166.0,zum,1.0,face scrub
1300,B07P8ZN5JS,𝗧𝗢𝗣 𝟮𝟬𝟮𝟯* hyaluronic acid face serum - 2.5% pu...,9.99,11.99,https://www.amazon.com/dp/B07P8ZN5JS,4.3,24024.0,eclat skincare,1.0,serum


In [29]:
#product_df1=product_df1.drop(1181)

In [82]:
'''
The original column datatypes:

ASIN                object
product_name        object
price               object
listPrice           object
detailPageURL       object
rating             float64
totalReviews       float64
product_company     object
isPrimeEligible    float64
Category            object
dtype: object

'''

print(product_df1.dtypes)

ASIN                object
product_name        object
price              float64
listPrice          float64
detailPageURL       object
rating             float64
totalReviews       float64
product_company     object
isPrimeEligible    float64
Category            object
dtype: object


In [ ]:
'''
INGREDIENT AND TAG COLLECTION THROUGH BEAUTIFUL SOUP

In this section, BeautifulSoup will be used to scrape the incidecoder.com website to obtain the ingredients of the products
mentioned in the dataset. For this, the relevant libraries were be installed and a get request was be sent to the product 
page. If the request was successfull the page content was parsed & the html sructure of the page was inspected for two 
sections: "Ingredients" and "Highlights". The text content of these sections were extracted and printed. 
'''
!pip install beautifulsoup4
!pip install requests

In [83]:
import requests
from bs4 import BeautifulSoup

# URL of the product page
url = "https://incidecoder.com/products/olay-total-effects-7-in-one-anti-aging-transforming-eye-cream"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the ingredients section by inspecting the HTML structure of the page
    ingredients_section = soup.find("div", class_="showmore-section ingredlist-short-like-section", id="showmore-section-ingredlist-short")
    
    # Find the highlights section by inspecting the HTML structure of the page
    highlights_section = soup.find("div", class_="ingredlist-by-function-block")
    
    # Extract the text content of the ingredients and highlights sections
    ingredients_text = ingredients_section.get_text() if ingredients_section else "Ingredients not found"
    highlights_text = highlights_section.get_text() if highlights_section else "Highlights not found"
    
    # Print the extracted information
    print("Ingredients:")
    print(ingredients_text)
    print("\nHighlights:")
    print(highlights_text)
else:
    print(f"Failed to retrieve the page (status code {response.status_code})")

Ingredients:

Water,
                            Glycerin,
                            Petrolatum,
                            Isopropyl Isostearate,
                            Isohexadecane,
                            Niacinamide*,
                            Stearyl Alcohol,
                            Dimethicone,
                            Cetyl Alcohol,
                            
        [more]Panthenol**,
                            Tocopheryl Acetate***,
                            Sodium Ascorbyl Phosphate^,
                            Retinyl Propionate^^,
                            Camellia Sinensis Leaf Extract^*,
                            BHT^**,
                            Cucumis Sativus (Cucumber) Fruit Extract,
                            C13-14 Isoparaffin,
                            Caprylyl Glycol,
                            Dimethiconol,
                            Cetearyl Glucoside,
                            Cetearyl Alcohol,
                          

In [84]:
# Initialize lists to store ingredients and highlights
ingredients_list = []
highlights_list = []

# Iterate through each row in the DataFrame
for index, row in product_df1.iterrows():
    # Get the product name from the current row
    product_name = row['product_name']
    
    # Construct the URL for the product page using the product name
    url = f"https://incidecoder.com/products/{product_name.replace(' ', '-')}"
    
    # Send an HTTP GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Find the ingredients section by inspecting the HTML structure of the page
        ingredients_section = soup.find("div", class_="showmore-section ingredlist-short-like-section", id="showmore-section-ingredlist-short")
        
        # Find the highlights section by inspecting the HTML structure of the page
        highlights_section = soup.find("div", class_="ingredlist-by-function-block")
        
        # Extract the text content of the ingredients and highlights sections
        ingredients_text = ingredients_section.get_text() if ingredients_section else "Ingredients not found"
        highlights_text = highlights_section.get_text() if highlights_section else "Highlights not found"
        
        # Append the ingredients and highlights to the respective lists
        ingredients_list.append(ingredients_text)
        highlights_list.append(highlights_text)
    else:
        # Handle the case where the page couldn't be retrieved
        ingredients_list.append("Not found")
        highlights_list.append("Not found")

# Add new columns for ingredients and highlights to the DataFrame
product_df1['ingredients'] = ingredients_list
product_df1['highlights'] = highlights_list

C:\Users\vakit\AppData\Local\Temp\ipykernel_20428\1644441028.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_df1['ingredients'] = ingredients_list
C:\Users\vakit\AppData\Local\Temp\ipykernel_20428\1644441028.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_df1['highlights'] = highlights_list


In [88]:
final_dataset=product_df1.copy(deep=True)
final_dataset.to_csv(r'final_dataset.csv', index=False)